**Import Required Libraries**

In [0]:
from pyspark.sql import functions as F

**Define start and end dates**

In [0]:
# start date and end date
start_date = "2024-01-01"
end_date   = "2025-12-01"

In [0]:
# 1️⃣ Generate one row per month start between start_date and end_date
df = (
    spark.sql(f"""
        SELECT explode(
            sequence(
                to_date('{start_date}'),
                to_date('{end_date}'),
                interval 1 month
            )
        ) AS month_start_date
    """)
)


# 2️⃣ Add useful analytics columns
df = (
    df
    # Surrogate key at month grain
    .withColumn("date_key", F.date_format("month_start_date", "yyyyMM").cast("int"))
    .withColumn("year", F.year("month_start_date"))
    .withColumn("month_name", F.date_format("month_start_date", "MMMM"))
    .withColumn("month_short_name", F.date_format("month_start_date", "MMM"))
    .withColumn("quarter", F.concat(F.lit("Q"), F.quarter("month_start_date")))
    .withColumn("year_quarter", F.concat(F.col("year"), F.lit("-Q"), F.quarter("month_start_date")))
)

In [0]:
display(df)

month_start_date,date_key,year,month_name,month_short_name,quarter,year_quarter
2024-01-01,202401,2024,January,Jan,Q1,2024-Q1
2024-02-01,202402,2024,February,Feb,Q1,2024-Q1
2024-03-01,202403,2024,March,Mar,Q1,2024-Q1
2024-04-01,202404,2024,April,Apr,Q2,2024-Q2
2024-05-01,202405,2024,May,May,Q2,2024-Q2
2024-06-01,202406,2024,June,Jun,Q2,2024-Q2
2024-07-01,202407,2024,July,Jul,Q3,2024-Q3
2024-08-01,202408,2024,August,Aug,Q3,2024-Q3
2024-09-01,202409,2024,September,Sep,Q3,2024-Q3
2024-10-01,202410,2024,October,Oct,Q4,2024-Q4


In [0]:
# Save as a table
df.write \
    .mode("overwrite") \
    .format("delta") \
    .saveAsTable("fmcg.gold.dim_date")